In [ ]:
import os, sys
sys.path.append(os.getcwd())
import time
import numpy as np
import tensorflow as tf
import collections
import re
import locale
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pickle5 as pickle
import gc

In [ ]:
locale.setlocale(locale.LC_ALL, '')

_params = {}
_param_aliases = {}
def param(name, *args, **kwargs):
    if name not in _params:
        kwargs['name'] = name
        param = tf.Variable(*args, **kwargs)
        param.param = True
        _params[name] = param
    result = _params[name]
    i = 0
    while result in _param_aliases:
        i += 1
        result = _param_aliases[result]
    return result

def params_with_name(name):
    return [p for n,p in _params.items() if name in n]

def delete_all_params():
    _params.clear()

def alias_params(replace_dict):
    for old,new in replace_dict.items():
        _param_aliases[old] = new

def delete_param_aliases():
    _param_aliases.clear()

def print_model_settings(locals_):
    print("Uppercase local vars:")
    all_vars = [(k,v) for (k,v) in locals_.items() if (k.isupper() and k!='T' and k!='SETTINGS' and k!='ALL_SETTINGS')]
    all_vars = sorted(all_vars, key=lambda x: x[0])
    for var_name, var_value in all_vars:
        print("\t{}: {}".format(var_name, var_value))


def print_model_settings_dict(settings):
    print("Settings dict:")
    all_vars = [(k,v) for (k,v) in settings.items()]
    all_vars = sorted(all_vars, key=lambda x: x[0])
    for var_name, var_value in all_vars:
        print("\t{}: {}".format(var_name, var_value))

In [ ]:
_since_beginning = collections.defaultdict(lambda: {})
_since_last_flush = collections.defaultdict(lambda: {})

_iter = [0]
def tick():
    _iter[0] += 1

def plot(name, value):
    _since_last_flush[name][_iter[0]] = value

def flush():
    prints = []

    for name, vals in _since_last_flush.items():
        prints.append("{}\t{}".format(name, np.mean(list(vals.values()))))
        _since_beginning[name].update(vals)

        x_vals = np.sort(list(_since_beginning[name].keys()))
        y_vals = [_since_beginning[name][x] for x in x_vals]

        plt.clf()
        plt.plot(x_vals, y_vals)
        plt.xlabel('iteration')
        plt.ylabel(name)
        plt.savefig(name.replace(' ', '_test')+'.jpg')

    print("iter {}\t{}".format(_iter[0], "\t".join(prints)))
    _since_last_flush.clear()

    with open('log_test.pkl', 'wb') as f:
        pickle.dump(dict(_since_beginning), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
_default_weightnorm = False
def enable_default_weightnorm():
    global _default_weightnorm
    _default_weightnorm = True

def Conv1D(name, input_dim, output_dim, filter_size, inputs, stride=1, gain=1.):
    """
    inputs: tensor of shape (batch size, num channels, width)
    mask_type: one of None, 'a', 'b'

    returns: tensor of shape (batch size, num channels, width)
    """
    with tf.name_scope(name) as scope:

        def uniform(stdev, size):
            return np.random.uniform(
                low=-stdev * np.sqrt(3),
                high=stdev * np.sqrt(3),
                size=size
            ).astype('float32')

        fan_in = input_dim * filter_size
        fan_out = output_dim * filter_size / stride
        filters_stdev = np.sqrt(2./(fan_in+fan_out))

        filter_values = uniform(
            filters_stdev,
            (filter_size, input_dim, output_dim)
        )
        filter_values *= gain

        filters = param(name+'.Filters', filter_values)

        result = tf.nn.conv1d(
            inputs, 
            filters=filters, 
            stride=stride,
            padding='SAME',
            data_format='NCW'
        )

        _biases = param(name+'.Biases', np.zeros([output_dim], dtype='float32'))

        result = tf.expand_dims(result, 3)
        result = tf.nn.bias_add(result, _biases, data_format='NCW')
        result = tf.squeeze(result)

        return result

In [ ]:
_default_weightnorm = False
def enable_default_weightnorm():
    global _default_weightnorm
    _default_weightnorm = True

def disable_default_weightnorm():
    global _default_weightnorm
    _default_weightnorm = False

_weights_stdev = None
def set_weights_stdev(weights_stdev):
    global _weights_stdev
    _weights_stdev = weights_stdev

def unset_weights_stdev():
    global _weights_stdev
    _weights_stdev = None

def Linear(
        name, 
        input_dim, 
        output_dim, 
        inputs,
        biases=True,
        initialization=None,
        weightnorm=None,
        gain=1.
        ):
    """
    initialization: None, `lecun`, 'glorot', `he`, 'glorot_he', `orthogonal`, `("uniform", range)`
    """
    with tf.name_scope(name) as scope:

        def uniform(stdev, size):
            if _weights_stdev is not None:
                stdev = _weights_stdev
            return np.random.uniform(
                low=-stdev * np.sqrt(3),
                high=stdev * np.sqrt(3),
                size=size
            ).astype('float32')
        
        def sample(shape):
            if len(shape) < 2:
                raise RuntimeError("Only shapes of length 2 or more are "
                                   "supported.")
            flat_shape = (shape[0], np.prod(shape[1:]))
             # TODO: why normal and not uniform?
            a = np.random.normal(0.0, 1.0, flat_shape)
            u, _, v = np.linalg.svd(a, full_matrices=False)
            # pick the one with the correct shape
            q = u if u.shape == flat_shape else v
            q = q.reshape(shape)
            return q.astype('float32')
        
        weight_values = sample((input_dim, output_dim))

        weight_values *= gain

        weight = param(
            name + '.W',
            weight_values
        )
        if inputs.get_shape().ndims == 2:
            result = tf.matmul(inputs, weight)
        else:
            reshaped_inputs = tf.reshape(inputs, [-1, input_dim])
            result = tf.matmul(reshaped_inputs, weight)
            result = tf.reshape(result, tf.pack(tf.unpack(tf.shape(inputs))[:-1] + [output_dim]))

        if biases:
            result = tf.nn.bias_add(
                result,
                param(
                    name + '.b',
                    np.zeros((output_dim,), dtype='float32')
                )
            )

        return result

In [ ]:
def tokenize_string(sample):
    return tuple(sample.lower().split(' '))

class NgramLanguageModel(object):
    def __init__(self, n, samples, tokenize=False):
        if tokenize:
            tokenized_samples = []
            for sample in samples:
                tokenized_samples.append(tokenize_string(sample))
            samples = tokenized_samples

        self._n = n
        self._samples = samples
        self._ngram_counts = collections.defaultdict(int)
        self._total_ngrams = 0
        for ngram in self.ngrams():
            self._ngram_counts[ngram] += 1
            self._total_ngrams += 1

    def ngrams(self):
        n = self._n
        for sample in self._samples:
            for i in range(len(sample)-n+1):
                yield sample[i:i+n]

    def unique_ngrams(self):
        return set(self._ngram_counts.keys())

    def log_likelihood(self, ngram):
        if ngram not in self._ngram_counts:
            return -np.inf
        else:
            return np.log(self._ngram_counts[ngram]) - np.log(self._total_ngrams)

    def kl_to(self, p):
        # p is another NgramLanguageModel
        log_likelihood_ratios = []
        for ngram in p.ngrams():
            log_likelihood_ratios.append(p.log_likelihood(ngram) - self.log_likelihood(ngram))
        return np.mean(log_likelihood_ratios)

    def cosine_sim_with(self, p):
        # p is another NgramLanguageModel
        p_dot_q = 0.
        p_norm = 0.
        q_norm = 0.
        for ngram in p.unique_ngrams():
            p_i = np.exp(p.log_likelihood(ngram))
            q_i = np.exp(self.log_likelihood(ngram))
            p_dot_q += p_i * q_i
            p_norm += p_i**2
        for ngram in self.unique_ngrams():
            q_i = np.exp(self.log_likelihood(ngram))
            q_norm += q_i**2
        return p_dot_q / (np.sqrt(p_norm) * np.sqrt(q_norm))

    def precision_wrt(self, p):
        # p is another NgramLanguageModel
        num = 0.
        denom = 0
        p_ngrams = p.unique_ngrams()
        for ngram in self.unique_ngrams():
            if ngram in p_ngrams:
                num += self._ngram_counts[ngram]
            denom += self._ngram_counts[ngram]
        return float(num) / denom

    def recall_wrt(self, p):
        return p.precision_wrt(self)

    def js_with(self, p):
        log_p = np.array([p.log_likelihood(ngram) for ngram in p.unique_ngrams()])
        log_q = np.array([self.log_likelihood(ngram) for ngram in p.unique_ngrams()])
        log_m = np.logaddexp(log_p - np.log(2), log_q - np.log(2))
        kl_p_m = np.sum(np.exp(log_p) * (log_p - log_m))

        log_p = np.array([p.log_likelihood(ngram) for ngram in self.unique_ngrams()])
        log_q = np.array([self.log_likelihood(ngram) for ngram in self.unique_ngrams()])
        log_m = np.logaddexp(log_p - np.log(2), log_q - np.log(2))
        kl_q_m = np.sum(np.exp(log_q) * (log_q - log_m))

        return 0.5*(kl_p_m + kl_q_m) / np.log(2)

def load_dataset(max_length, max_n_examples, tokenize=False, max_vocab_size=2048, data_dir=''):
    print("loading dataset...")
    lines = []
    finished = False

    c = 0
    with open(data_dir, 'r', encoding='latin1') as f:
        for line in f:
            c += 1
            line = line[:-1]
            if len(line) <= max_length:
                lines.append(line + ("`" * (max_length-len(line))))

    np.random.shuffle(lines)

    counts = collections.Counter(char for line in lines for char in line)

    charmap = {'unk':0}
    inv_charmap = ['unk']

    for char,count in counts.most_common(max_vocab_size-1):
        if char not in charmap:
            charmap[char] = len(inv_charmap)
            inv_charmap.append(char)

    filtered_lines = []
    for line in lines:
        filtered_line = []
        for char in line:
            if char in charmap:
                filtered_line.append(char)
            else:
                filtered_line.append('unk')
        filtered_lines.append(tuple(filtered_line))

    for i in range(10):
        print(filtered_lines[i])

    print("loaded {} lines in dataset-test".format(len(lines)))
    return filtered_lines, charmap, inv_charmap

In [ ]:
tf.compat.v1.disable_eager_execution()

# Download Google Billion Word at http://www.statmt.org/lm-benchmark/ and
# fill in the path to the extracted files here!
DATA_DIR = 'train_rock_you.txt'
if len(DATA_DIR) == 0:
    raise Exception('Please specify path to data directory in gan_language.py!')

BATCH_SIZE = 512 # Batch size
ITERS = 200000 # How many iterations to train for
SEQ_LEN = 10 # Sequence length in characters
DIM = 128 # Model dimensionality. This is fairly slow and overfits, even on
          # Billion Word. Consider decreasing for smaller datasets.
CRITIC_ITERS = 10 # How many critic iterations per generator iteration. We
                  # use 10 for the results in the paper, but 5 should work fine
                  # as well.
LAMBDA = 10 # Gradient penalty lambda hyperparameter.
MAX_N_EXAMPLES = 10000000 # Max number of data examples to load. If data loading
                          # is too slow or takes too much RAM, you can decrease
                          # this (at the expense of having less training data).
#195313
NUM_GENERATORS = 125
print_model_settings(locals().copy())

lines, charmap, inv_charmap = load_dataset(
    max_length=SEQ_LEN,
    max_n_examples=MAX_N_EXAMPLES,
    data_dir=DATA_DIR
)

Uppercase local vars:
	BATCH_SIZE: 512
	CRITIC_ITERS: 10
	DATA_DIR: train_rock_you.txt
	DIM: 128
	ITERS: 200000
	LAMBDA: 10
	MAX_N_EXAMPLES: 10000000
	NUM_GENERATORS: 125
	SEQ_LEN: 10
loading dataset...
('9', '9', '9', '6', '9', '3', '`', '`', '`', '`')
('j', 'c', '5', '8', '8', '8', '`', '`', '`', '`')
('y', 'o', 'u', 'a', 'n', 'd', 'i', '1', '3', '`')
('3', '5', '1', '1', '`', '`', '`', '`', '`', '`')
('2', '5', '2', '1', '9', '9', '9', '9', '`', '`')
('n', 'c', 'b', 'y', '3', 'h', '9', '5', '`', '`')
('c', 'l', 'i', 't', 's', '`', '`', '`', '`', '`')
('m', 'i', 'z', 'z', 'o', 'u', '8', '1', '`', '`')
('a', 't', '4', '2', '0', '8', '`', '`', '`', '`')
('h', 'a', 'm', 'b', 'o', 'n', 'e', '9', '`', '`')
loaded 7911499 lines in dataset-test


In [ ]:
def softmax(logits):
    return tf.reshape(
        tf.nn.softmax(
            tf.reshape(logits, [-1, len(charmap)])
        ),
        tf.shape(logits)
    )

def make_noise(shape):
    return tf.random.normal(shape)

def ResBlock(name, inputs):
    output = inputs
    output = tf.nn.relu(output)
    output = Conv1D(name+'.1', DIM, DIM, 5, output)
    output = tf.nn.relu(output)
    output = Conv1D(name+'.2', DIM, DIM, 5, output)
    return inputs + (0.3*output)

def Generator(n_samples, prev_outputs=None):
    output = make_noise(shape=[n_samples, 128])
    output = Linear('Generator.Input', 128, SEQ_LEN*DIM, output)
    output = tf.reshape(output, [-1, DIM, SEQ_LEN])
    output = ResBlock('Generator.1', output)
    output = ResBlock('Generator.2', output)
    output = ResBlock('Generator.3', output)
    output = ResBlock('Generator.4', output)
    output = ResBlock('Generator.5', output)
    output = Conv1D('Generator.Output', DIM, len(charmap), 1, output)
    output = tf.transpose(output, [0, 2, 1])
    output = softmax(output)
    return output

def Discriminator(inputs):
    output = tf.transpose(inputs, [0,2,1])
    output = Conv1D('Discriminator.Input', len(charmap), DIM, 1, output)
    output = ResBlock('Discriminator.1', output)
    output = ResBlock('Discriminator.2', output)
    output = ResBlock('Discriminator.3', output)
    output = ResBlock('Discriminator.4', output)
    output = ResBlock('Discriminator.5', output)
    output = tf.reshape(output, [-1, SEQ_LEN*DIM])
    output = Linear('Discriminator.Output', SEQ_LEN*DIM, 1, output)
    return output

In [ ]:
real_inputs_discrete = tf.compat.v1.placeholder(tf.int32, shape=[BATCH_SIZE, SEQ_LEN])
real_inputs = tf.one_hot(real_inputs_discrete, len(charmap))
fake_inputs = Generator(BATCH_SIZE)
fake_inputs_discrete = tf.argmax(fake_inputs, fake_inputs.get_shape().ndims-1)

disc_real = Discriminator(real_inputs)
disc_fake = Discriminator(fake_inputs)

disc_cost = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)
gen_cost = -tf.reduce_mean(disc_fake)

# WGAN lipschitz-penalty
alpha = tf.random.uniform(
    shape=[BATCH_SIZE,1,1], 
    minval=0.,
    maxval=1.
)
differences = fake_inputs - real_inputs
interpolates = real_inputs + (alpha*differences)
gradients = tf.gradients(Discriminator(interpolates), [interpolates])[0]
slopes = tf.sqrt(
    tf.compat.v1.reduce_sum(
        tf.square(gradients),
        reduction_indices=[1,2]
    )
)
gradient_penalty = tf.reduce_mean((slopes-1.)**2)
disc_cost += LAMBDA*gradient_penalty

gen_params = params_with_name('Generator')
disc_params = params_with_name('Discriminator')

gen_train_op = tf.compat.v1.train.AdamOptimizer(
    learning_rate=1e-4,
    beta1=0.5,
    beta2=0.9
).minimize(gen_cost, var_list=gen_params)
disc_train_op = tf.compat.v1.train.AdamOptimizer(
    learning_rate=1e-4,
    beta1=0.5,
    beta2=0.9
).minimize(disc_cost, var_list=disc_params)

# During training we monitor JS divergence between the true & generated ngram
# distributions for n=1,2,3,4. To get an idea of the optimal values, we
# evaluate these statistics on a held-out set first.
true_char_ngram_lms = [NgramLanguageModel(i+1, lines[10*BATCH_SIZE:], tokenize=False) for i in range(4)]
validation_char_ngram_lms = [NgramLanguageModel(i+1, lines[:10*BATCH_SIZE], tokenize=False) for i in range(4)]
for i in range(4):
    print("validation set JSD for n={}: {}".format(i+1, true_char_ngram_lms[i].js_with(validation_char_ngram_lms[i])))
true_char_ngram_lms = [NgramLanguageModel(i+1, lines, tokenize=False) for i in range(4)]

validation set JSD for n=1: 0.0005057924480176116
validation set JSD for n=2: 0.01729227208283827
validation set JSD for n=3: 0.14380669638211904
validation set JSD for n=4: 0.44478439693882094


In [ ]:
# Dataset iterator
def inf_train_gen():
    while True:
        np.random.shuffle(lines)
        for i in range(0, len(lines)-BATCH_SIZE+1, BATCH_SIZE):
            yield np.array(
                [[charmap[c] for c in l] for l in lines[i:i+BATCH_SIZE]], 
                dtype='int32'
            )

In [ ]:
def model_save(epoch):
    with open('model_test' + str(epoch) + '.pickle', 'wb') as f:
        pickle.dump(_params, f, pickle.HIGHEST_PROTOCOL)

def model_load(epoch):
    with open('model_e' + str(epoch) + '.pickle', 'rb') as f:
        return pickle.load(f)

In [ ]:
with tf.compat.v1.Session() as session:
    session.run(tf.compat.v1.initialize_all_variables())
#     new_saver = tf.compat.v1.train.import_meta_graph('model_e40000.meta')
#     new_saver.restore(session, 'model_e40000')
    saver = tf.compat.v1.train.Saver(max_to_keep=100)

    def generate_samples():
        samples = session.run(Generator(800000))
        samples = np.argmax(samples, axis=2)
        decoded_samples = []
        for i in range(len(samples)):
            decoded = []
            for j in range(len(samples[i])):
                decoded.append(inv_charmap[samples[i][j]])
            decoded_samples.append(tuple(decoded))
        return decoded_samples
    
    def generate_samples_for_lm():
        samples = session.run(fake_inputs)
        samples = np.argmax(samples, axis=2)
        decoded_samples = []
        for i in range(len(samples)):
            decoded = []
            for j in range(len(samples[i])):
                decoded.append(inv_charmap[samples[i][j]])
            decoded_samples.append(tuple(decoded))
        return decoded_samples
    
    gen = inf_train_gen()
    
    for iteration in range(ITERS):
        start_time = time.time()
        # Train generator
        if iteration > 0:
            _ = session.run(gen_train_op)

        # Train critic
        for i in range(CRITIC_ITERS):
            _data = next(gen)
            _disc_cost, _ = session.run(
                [disc_cost, disc_train_op],
                feed_dict={real_inputs_discrete:_data}
            )
        plot('time', time.time() - start_time)
        plot('train disc cost', _disc_cost)
        itter_list = [5000, 15000, 25000, 35000, 45000, 55000, 65000, 75000,85000,95000,105000,115000,125000,135000,145000,155000,165000,175000,185000,195000,199000]
        if iteration in itter_list:
            print("generating samples...")
            
            count = 0
            for i in range(NUM_GENERATORS):
                samples_to_output = []
                samples_to_output.extend(generate_samples())
                count+=len(samples_to_output)
                print(f"generated samples. Total samplse: {count}")
                with open('samples_new_append{}.txt'.format(iteration), 'a') as f:
                    for s in samples_to_output:
                        s = "".join(s)
                        f.write(s + "\n")
            print(f"generated {len(samples_to_output)} passwords and wrote to samples_new{iteration}.txt")
            del samples_to_output
            gc.collect()
            
        if iteration % 1000 == 999:
            print("updating js")
            samples = []           
            samples.extend(generate_samples_for_lm())
            for i in range(4):
                lm = NgramLanguageModel(i+1, samples, tokenize=False)
                plot('js{}'.format(i+1), lm.js_with(true_char_ngram_lms[i]))
            print("done updating")
            del samples
            gc.collect()
                    
        if iteration % 5000 == 0:
            saver.save(session, save_path='model_e_ap'+str(iteration))
            #model_save(iteration)

        if iteration % 1000 == 999:
            flush()
        
        tick()


Instructions for updating:
Use `tf.global_variables_initializer` instead.
updating js
done updating
iter 999	time	0.22695676374435425	train disc cost	-1.6192326545715332	js1	0.19662795334459418	js2	0.5393857276560634	js3	0.7923199243571465	js4	0.9121388352786571
updating js
done updating
iter 1999	time	0.23030275368690492	train disc cost	-1.2153129577636719	js1	0.09275882206502745	js2	0.326192865960691	js3	0.6115212270247324	js4	0.8119258219776365
updating js
done updating
iter 2999	time	0.2281457531452179	train disc cost	-1.1255908012390137	js1	0.09096288763492309	js2	0.2855822535151476	js3	0.5785764568445074	js4	0.7994168118259081
updating js
done updating
iter 3999	time	0.2291683831214905	train disc cost	-1.0603646039962769	js1	0.06571662837272398	js2	0.2141268251988978	js3	0.4994614665056671	js4	0.7668251170649912
updating js
done updating
iter 4999	time	0.22877079510688783	train disc cost	-0.9161466360092163	js1	0.04893308955207843	js2	0.1618709823119402	js3	0.4718482461857245	js4

updating js
done updating
iter 13999	time	0.22691228246688844	train disc cost	-0.7335512042045593	js1	0.03816071336242317	js2	0.13434715477732787	js3	0.43612631260416623	js4	0.7468671238633171
updating js
done updating
iter 14999	time	0.22659291052818298	train disc cost	-0.7289426326751709	js1	0.03831852400936937	js2	0.1317394688743505	js3	0.43541382819187996	js4	0.746032144012534
generating samples...
generated samples. Total samplse: 800000
generated samples. Total samplse: 1600000
generated samples. Total samplse: 2400000
generated samples. Total samplse: 3200000
generated samples. Total samplse: 4000000
generated samples. Total samplse: 4800000
generated samples. Total samplse: 5600000
generated samples. Total samplse: 6400000
generated samples. Total samplse: 7200000
generated samples. Total samplse: 8000000
generated samples. Total samplse: 8800000
generated samples. Total samplse: 9600000
generated samples. Total samplse: 10400000
generated samples. Total samplse: 11200000
gener

generated samples. Total samplse: 9600000
generated samples. Total samplse: 10400000
generated samples. Total samplse: 11200000
generated samples. Total samplse: 12000000
generated samples. Total samplse: 12800000
generated samples. Total samplse: 13600000
generated samples. Total samplse: 14400000
generated samples. Total samplse: 15200000
generated samples. Total samplse: 16000000
generated samples. Total samplse: 16800000
generated samples. Total samplse: 17600000
generated samples. Total samplse: 18400000
generated samples. Total samplse: 19200000
generated samples. Total samplse: 20000000
generated samples. Total samplse: 20800000
generated samples. Total samplse: 21600000
generated samples. Total samplse: 22400000
generated samples. Total samplse: 23200000
generated samples. Total samplse: 24000000
generated samples. Total samplse: 24800000
generated samples. Total samplse: 25600000
generated samples. Total samplse: 26400000
generated samples. Total samplse: 27200000
generated sa

generated samples. Total samplse: 25600000
generated samples. Total samplse: 26400000
generated samples. Total samplse: 27200000
generated samples. Total samplse: 28000000
generated samples. Total samplse: 28800000
generated samples. Total samplse: 29600000
generated samples. Total samplse: 30400000
generated samples. Total samplse: 31200000
generated samples. Total samplse: 32000000
generated samples. Total samplse: 32800000
generated samples. Total samplse: 33600000
generated samples. Total samplse: 34400000
generated samples. Total samplse: 35200000
generated samples. Total samplse: 36000000
generated samples. Total samplse: 36800000
generated samples. Total samplse: 37600000
generated samples. Total samplse: 38400000
generated samples. Total samplse: 39200000
generated samples. Total samplse: 40000000
generated samples. Total samplse: 40800000
generated samples. Total samplse: 41600000
generated samples. Total samplse: 42400000
generated samples. Total samplse: 43200000
generated s

generated samples. Total samplse: 41600000
generated samples. Total samplse: 42400000
generated samples. Total samplse: 43200000
generated samples. Total samplse: 44000000
generated samples. Total samplse: 44800000
generated samples. Total samplse: 45600000
generated samples. Total samplse: 46400000
generated samples. Total samplse: 47200000
generated samples. Total samplse: 48000000
generated samples. Total samplse: 48800000
generated samples. Total samplse: 49600000
generated samples. Total samplse: 50400000
generated samples. Total samplse: 51200000
generated samples. Total samplse: 52000000
generated samples. Total samplse: 52800000
generated samples. Total samplse: 53600000
generated samples. Total samplse: 54400000
generated samples. Total samplse: 55200000
generated samples. Total samplse: 56000000
generated samples. Total samplse: 56800000
generated samples. Total samplse: 57600000
generated samples. Total samplse: 58400000
generated samples. Total samplse: 59200000
generated s

...

generated samples. Total samplse: 63200000
generated samples. Total samplse: 64000000
generated samples. Total samplse: 64800000
generated samples. Total samplse: 65600000
generated samples. Total samplse: 66400000
generated samples. Total samplse: 67200000
generated samples. Total samplse: 68000000
generated samples. Total samplse: 68800000
generated samples. Total samplse: 69600000
generated samples. Total samplse: 70400000
generated samples. Total samplse: 71200000
generated samples. Total samplse: 72000000
generated samples. Total samplse: 72800000
generated samples. Total samplse: 73600000
generated samples. Total samplse: 74400000
generated samples. Total samplse: 75200000
generated samples. Total samplse: 76000000
generated samples. Total samplse: 76800000
generated samples. Total samplse: 77600000
generated samples. Total samplse: 78400000
generated samples. Total samplse: 79200000
generated samples. Total samplse: 80000000
generated samples. Total samplse: 80800000
generated s

generated samples. Total samplse: 78400000
generated samples. Total samplse: 79200000
generated samples. Total samplse: 80000000
generated samples. Total samplse: 80800000
generated samples. Total samplse: 81600000
generated samples. Total samplse: 82400000
generated samples. Total samplse: 83200000
generated samples. Total samplse: 84000000
generated samples. Total samplse: 84800000
generated samples. Total samplse: 85600000
generated samples. Total samplse: 86400000
generated samples. Total samplse: 87200000
generated samples. Total samplse: 88000000
generated samples. Total samplse: 88800000
generated samples. Total samplse: 89600000
generated samples. Total samplse: 90400000
generated samples. Total samplse: 91200000
generated samples. Total samplse: 92000000
generated samples. Total samplse: 92800000
generated samples. Total samplse: 93600000
generated samples. Total samplse: 94400000
generated samples. Total samplse: 95200000
generated samples. Total samplse: 96000000
generated s

generated samples. Total samplse: 93600000
generated samples. Total samplse: 94400000
generated samples. Total samplse: 95200000
generated samples. Total samplse: 96000000
generated samples. Total samplse: 96800000
generated samples. Total samplse: 97600000
generated samples. Total samplse: 98400000
generated samples. Total samplse: 99200000
generated samples. Total samplse: 100000000
generated 800000 passwords and wrote to samples_new175000.txt
updating js
done updating
iter 175999	time	0.21199053263664244	train disc cost	-0.5247135162353516	js1	0.018071314313021423	js2	0.09191299801816694	js3	0.39823426025069814	js4	0.7233756186595579
updating js
done updating
iter 176999	time	0.2156974275112152	train disc cost	-0.5240952372550964	js1	0.019144730400168394	js2	0.09095329022384559	js3	0.4021232137427342	js4	0.7252457234403806
updating js
done updating
iter 177999	time	0.2153084704875946	train disc cost	-0.5288135409355164	js1	0.01812702175135208	js2	0.09226159590271195	js3	0.4034140568

updating js
done updating
iter 187999	time	0.21242934322357176	train disc cost	-0.5264360904693604	js1	0.01866373129413744	js2	0.09421579664837886	js3	0.4029188911223741	js4	0.7230806880888118
updating js
done updating
iter 188999	time	0.2142292788028717	train disc cost	-0.5294938087463379	js1	0.017734039476398265	js2	0.09845331057867614	js3	0.41063708136098054	js4	0.7315064095762777
updating js
done updating
iter 189999	time	0.21407066893577575	train disc cost	-0.5273481011390686	js1	0.017860785096116372	js2	0.09482755722329435	js3	0.40481191337003963	js4	0.728365442342636
updating js
done updating
iter 190999	time	0.21436574006080628	train disc cost	-0.5253685116767883	js1	0.019487805342537877	js2	0.09879558753180477	js3	0.40933279939508604	js4	0.7290360108112268
updating js
done updating
iter 191999	time	0.21419033694267273	train disc cost	-0.5283514857292175	js1	0.019332460039300864	js2	0.09411251589271173	js3	0.39831828366405414	js4	0.7262579739595608
updating js
done updating
ite

generated samples. Total samplse: 8800000
generated samples. Total samplse: 9600000
generated samples. Total samplse: 10400000
generated samples. Total samplse: 11200000
generated samples. Total samplse: 12000000
generated samples. Total samplse: 12800000
generated samples. Total samplse: 13600000
generated samples. Total samplse: 14400000
generated samples. Total samplse: 15200000
generated samples. Total samplse: 16000000
generated samples. Total samplse: 16800000
generated samples. Total samplse: 17600000
generated samples. Total samplse: 18400000
generated samples. Total samplse: 19200000
generated samples. Total samplse: 20000000
generated samples. Total samplse: 20800000
generated samples. Total samplse: 21600000
generated samples. Total samplse: 22400000
generated samples. Total samplse: 23200000
generated samples. Total samplse: 24000000
generated samples. Total samplse: 24800000
generated samples. Total samplse: 25600000
generated samples. Total samplse: 26400000
generated sam